Clase 6
====
Hoy veremos herramientas avanzadas de cobrapy.

Objetivos

* Lectura de modelos metabólicos
* Flux Varional Analysis (FVA)
* Parsimonious FBA


Lectura de modelos metabólicos
===========================

Cobrapy puede leer y escribir modelos metabólicos en varios formatos, incluyendo SBML, JSON, YAML, MAT, y pickle. Hoy trabajaremos con SBML, el cual es el formato preferido. 

Los modelos metabólicos para varias especies pueden ser descargados [aquí](http://systemsbiology.ucsd.edu/Downloads). Para los ejercicios de hoy descarga el modelo [iMM904 de Saccharomyces cerevisiae](https://raw.githubusercontent.com/modcommet/Clases/master/iMM904.xml)


In [1]:
import cobra
model = cobra.io.read_sbml_model("iMM904.xml")
model

Name,iMM904
Memory address,0x07fa8c02bdd10
Number of metabolites,1226
Number of reactions,1577
Objective expression,-1.0*BIOMASS_SC5_notrace_reverse_93090 + 1.0*BIOMASS_SC5_notrace
Compartments,"cytosol, extracellular space, golgi apparatus, mitochondria, nucleus, endoplasmic reticulum, vacuole, peroxisome/glyoxysome"


Running Flux Vartional Analysis (FVA)
=====================================

En general FBA no tiene una solución única, pero varias que resultan en el mismo valor de la función objetivo. El análisis de variabilidad de flujo (FVA: Flux Variational Analysis) permite encontra rel rango de flujos de cada metabólito en el valor optimo.

In [13]:
# Primero encontramos la solución optima del modelo
solution = model.optimize()

In [16]:
# Luego computamos los rangos de flujo de cada metabólito
from cobra.flux_analysis import flux_variability_analysis

fva=flux_variability_analysis(model)

In [15]:
# Para ver los rangos de los primeros 10 metabolitos
fva[0:10]

,maximum,minimum
13BGH,-2.892555e-09,0.000000e+00
13BGHe,0.000000e+00,0.000000e+00
13GS,3.266700e-01,3.266700e-01
16GS,-1.446278e-09,0.000000e+00
23CAPPD,0.000000e+00,0.000000e+00
2DDA7Ptm,0.000000e+00,-7.608291e-02
2DHPtm,-1.709885e-09,0.000000e+00
2DOXG6PP,0.000000e+00,0.000000e+00
2HBO,0.000000e+00,-6.879816e-14
2HBt2,0.000000e+00,-6.879816e-14


parsimonious FBA
=========

Por otra parte el valor optimo encontrado por FBA no necesariamente implica que se están ocupando el minimo de flujos disponibles. Por ejemplo, cuando hay ciclos es posible que algunos flujos tomen valores arbitrariamente altos sin afectar el valor de la función objetivo. Esto puede ser una causa de artefactos. Para prevenir esto se puede usar la herramienta parsimoniouos FBA (pFBA), la cual realiza una optimización en dos niveles. En el nivel interna se optimiza la función objetivo, tal como en un FBA normal, mientras que en la optimización externa se minimiza la suma total de los flujos con la restricción de que se conserve el valor óptimo encontrado por la optimización interna.

Veamos un ejemplo:

In [20]:
# Este es un FBA regular
fba_solution = model.optimize()
# Ahora veamos como seria el pFBA
pfba_solution = cobra.flux_analysis.pfba(model)

In [21]:
# Los valores de las funciones objetivos son:
print fba_solution.f
print pfba_solution.f

0.287865703704
338.005991582


In [23]:
# Los valores de la reaccion de biomasa (funcion objetivo del FBA) deberia ser el mismo en ambos casos
print fba_solution.fluxes["BIOMASS_SC5_notrace"]
print pfba_solution.fluxes["BIOMASS_SC5_notrace"]
print abs(fba_solution.fluxes["BIOMASS_SC5_notrace"] - pfba_solution.fluxes["BIOMASS_SC5_notrace"])

0.287865703704
0.287865703704
6.10622663544e-16


In [24]:
# Sin embargo la suma de los flujos deberia ser menor en pFBA
print sum(fba_solution)
print sum(pfba_solution)

16000110.9572
134.311328913


Ejercicio
=========

1. Descargen una la read metabólica de Ecoli 
2. Escriban una función para realizar todos los análsis vistos en este clase
3. Escriban su función como un script.